## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-14-18-58-22 +0000,nyt,"Officials from Denmark, Greenland and the U.S....",https://www.nytimes.com/2026/01/14/world/europ...
1,2026-01-14-18-55-57 +0000,nyt,Judge Puts Off Ruling on Minnesota’s Request t...,https://www.nytimes.com/2026/01/14/us/minnesot...
2,2026-01-14-18-53-15 +0000,nypost,Wild dashcam footage shows speeding car erupt ...,https://nypost.com/2026/01/14/us-news/dashcam-...
3,2026-01-14-18-53-00 +0000,nyt,Supreme Court Sides With Conservative Congress...,https://www.nytimes.com/2026/01/14/us/politics...
4,2026-01-14-18-52-49 +0000,nypost,Frustration as 2028 LA Summer Olympics ticket ...,https://nypost.com/2026/01/14/us-news/2028-la-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2378/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,66
119,iran,39
713,china,19
13,ice,18
106,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
367,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...,143
268,2026-01-14-03-31-29 +0000,nypost,Trump threatens ‘very strong action’ against I...,https://nypost.com/2026/01/13/us-news/trump-th...,141
61,2026-01-14-17-25-00 +0000,nypost,Iran issues sickening assassination threat aga...,https://nypost.com/2026/01/14/world-news/iran-...,139
193,2026-01-14-11-32-01 +0000,cbc,Iran judicial chief says quick justice for pro...,https://www.cbc.ca/news/world/iran-protests-wh...,138
82,2026-01-14-16-54-05 +0000,nyt,U.S. Moves Some Personnel From Qatar Air Base ...,https://www.nytimes.com/2026/01/14/us/politics...,132


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
367,143,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...
166,78,2026-01-14-13-02-49 +0000,nypost,Doctors in Iran collapsing from exhaustion as ...,https://nypost.com/2026/01/14/world-news/iran-...
308,66,2026-01-14-00-11-20 +0000,wapo,New York governor will push for state lawsuits...,https://www.washingtonpost.com/nation/2026/01/...
30,58,2026-01-14-18-14-00 +0000,wsj,"Fed governor Stephen Miran, a Trump appointee,...",https://www.wsj.com/economy/central-banking/fe...
336,55,2026-01-13-22-37-28 +0000,nypost,Eric Adams tells heckler ‘go f–k yourself’ as ...,https://nypost.com/2026/01/13/us-news/eric-ada...
371,54,2026-01-13-20-51-30 +0000,cbc,"As Carney heads to China to talk trade, Ontari...",https://www.cbc.ca/news/canada/toronto/doug-fo...
376,49,2026-01-13-20-29-53 +0000,wapo,Prosecutors seek death for South Korea’s forme...,https://www.washingtonpost.com/world/2026/01/1...
88,48,2026-01-14-16-36-48 +0000,nypost,Home sales slump dragged through 2025 as mortg...,https://nypost.com/2026/01/14/business/home-sa...
157,46,2026-01-14-13-18-23 +0000,cbc,"Denmark, Greenland bring united front to White...",https://www.cbc.ca/news/world/white-house-meet...
9,38,2026-01-14-18-48-00 +0000,wsj,FBI agents searched the home of a Washington P...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
